# Imports

In [68]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [69]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.MinimaxAgent import MinimaxAgent

# Setup

In [70]:
np.random.seed(42)

EPISODES = 1000
MAX_DEPTH = 9999

# Agent Testing Function

In [71]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    unweighted_mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        """ AGENT VALIDATION """
        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        unweighted_mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, unweighted_mean_nodes

# Test running function

In [72]:
def run_tests(misere_agents, normal_agents, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    os.makedirs(save_path_prefix, exist_ok=True)

    print("-" * 60)
    classic_configuration = False

    if pile_count == 0 or max_pile == 0:
        classic_configuration = True
        initial_piles = np.tile([1, 3, 5, 7], (episodes, 1))

        print("Classic configuration used")
    else:
        initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))

        print(f"Configuration: pile_count: {pile_count}, max_pile: {max_pile}, max_depth: {max_depth}")

    print("-" * 60)
    misere_modes = np.random.choice([True, False], size=episodes)

    for agent_key in misere_agents.keys():
        if not classic_configuration:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-{pile_count}-{max_pile}.npz"
        else:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-classic.npz"

        if os.path.exists(filename):
            print(f"Skipping {filename}")
            continue

        misere_agent = misere_agents[agent_key]
        normal_agent = normal_agents[agent_key]

        results = test_agent(misere_agent, normal_agent, misere_modes, initial_piles)
        np.savez(filename, results)

# Plotting results

In [73]:
def load_all_results(vary_pile_counts, vary_max_piles,
                     max_depth,
                     folder="../savedData/Minimax"):
    records = []
    agents = ['n','c','p','a','cp','ca','pa','cpa']
    for ag in agents:
        for pc in vary_pile_counts:
            for mp in vary_max_piles:
                fn = f"{folder}/minimax-{ag}-{max_depth}-{pc}-{mp}.npz"
                if not os.path.exists(fn):
                    continue
                exp_nodes, moves_cnt, _ = np.load(fn)['arr_0']
                records.append({
                    'agent': ag,
                    'pile_count': pc,
                    'max_pile':   mp,
                    'nodes_per_move': exp_nodes.sum() / moves_cnt.sum(),
                    'nodes_per_game': exp_nodes.mean(),
                    'moves_per_game': moves_cnt.mean()
                })
    return pd.DataFrame(records)

def plot_results(vary_pile_counts, vary_max_piles,
                          max_depth,
                          folder="../savedData/Minimax",
                          save_folder="../savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)
    df = load_all_results(vary_pile_counts, vary_max_piles, max_depth, folder)

    ordered_agents = ['Normal','Canonical','P-pruning','Aggressive',
                  'Canon+P','Canon+A','P+A','All']

    name_map = {
        'n':'Normal','c':'Canonical','p':'P-pruning','a':'Aggressive',
        'cp':'Canon+P','ca':'Canon+A','pa':'P+A','cpa':'All'
    }
    df['agent_name'] = df['agent'].map(name_map)

    metrics = [
        ('nodes_per_move', 'Nodes per Move', 'Blues'),
        ('nodes_per_game', 'Nodes per Game', 'Greens'),
        ('moves_per_game', 'Moves per Game', 'Purples'),
    ]

    for col, title, cmap in metrics:
        vmin = df[col].replace(0, np.nan).min()  # avoid zeros
        vmax = df[col].max()
        norm = LogNorm(vmin=vmin, vmax=vmax)

        fig, axes = plt.subplots(2, 4, figsize=(16, 8), sharex=True, sharey=True)
        axes = axes.flatten()

        for ax, agent_name in zip(axes, ordered_agents):
            grp = df[df['agent_name'] == agent_name]
            pivot = grp.pivot(index='pile_count',
                              columns='max_pile',
                              values=col)

            sns.heatmap(pivot, annot=True, fmt=".1f", cbar=False,
                        cmap=cmap, ax=ax,
                        xticklabels=vary_max_piles, yticklabels=vary_pile_counts,
                        norm=norm,
                        annot_kws={"fontsize":6})
            ax.set_title(agent_name, fontsize=10)
            ax.set_xlabel('max_pile')
            ax.set_ylabel('pile_count')

        # add shared colorbar
        cbar_ax = fig.add_axes([.92, .3, .02, .4])
        norm = plt.Normalize(df[col].min(), df[col].max())
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        fig.colorbar(sm, cax=cbar_ax, label=title)

        fig.suptitle(f"{title} (depth={max_depth})", fontsize=14)
        plt.tight_layout(rect=[0,0,0.9,1])
        out = f"{save_folder}/{col}_d{max_depth}.png"
        fig.savefig(out, dpi=200)
        plt.close(fig)
        print(f"Saved: {out}")

# Testing on shallow depths

In [74]:
for max_depth in [1, 2]:
    misere_agents = {
        'n': MinimaxAgent(misere=True, max_depth=max_depth),
        'c': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=True, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    normal_agents = {
        'n': MinimaxAgent(misere=False, max_depth=max_depth),
        'c': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=False, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    for pile_count in [2, 4, 8]:
        for max_count in [63, 127, 255]:
            run_tests(misere_agents, normal_agents, max_depth, pile_count, max_count, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 63, max_depth: 1
------------------------------------------------------------


100%|██████████| 1000/1000 [00:00<00:00, 1405.86it/s]


average explored nodes per move:      19.68
average explored nodes per game:     383.49
average moves per game:               19.49


100%|██████████| 1000/1000 [00:00<00:00, 1817.57it/s]


average explored nodes per move:      19.68
average explored nodes per game:     383.49
average moves per game:               19.49


...

average explored nodes per move:   41246.62
average explored nodes per game:  659120.94
average moves per game:               15.98


100%|██████████| 1000/1000 [32:25<00:00,  1.95s/it]


average explored nodes per move:   43925.07
average explored nodes per game:  649915.40
average moves per game:               14.80


100%|██████████| 1000/1000 [30:34<00:00,  1.83s/it]

average explored nodes per move:   43784.45
average explored nodes per game:  648622.85
average moves per game:               14.81


In [75]:
# for max_depth in [1, 2]:
#     plot_results(
#         vary_pile_counts=[2, 4, 8],
#         vary_max_piles=[63, 127, 255],
#         max_depth=max_depth,
#     )

/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_move_d1.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_game_d1.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


...

/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_move_d2.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_game_d2.png
Saved: ../savedPlots/Minimax/moves_per_game_d2.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


# Max depth testing on a smaller configuration

In [78]:
misere_agents = {
    'n': MinimaxAgent(misere=True, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

normal_agents = {
    'n': MinimaxAgent(misere=False, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

for pile_count in [2, 3, 4]:
    for max_pile in [3, 4, 5]:
        run_tests(misere_agents, normal_agents, MAX_DEPTH, pile_count, max_pile, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 3, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-c-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-p-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-a-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cp-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-ca-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-pa-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cpa-9999-2-3.npz
------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 4, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-c-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-p-9999-2-4.npz
Skipping ../savedData/Minimax/minimax-a-9999-2-4.n

In [77]:
# plot_results(
#     vary_pile_counts=[2, 3, 4],
#     vary_max_piles=[3, 4, 5],
#     max_depth=MAX_DEPTH
# )

/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_move_d9999.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])


Saved: ../savedPlots/Minimax/nodes_per_game_d9999.png
Saved: ../savedPlots/Minimax/moves_per_game_d9999.png


/var/folders/wz/c3crfx495cq6412hrdzsvv0r0000gn/T/ipykernel_20346/3123813282.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0,0,0.9,1])
